In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0819 08:30:30.391682  3912 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 08:30:30.396646  3912 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 08:30:30.397644  3912 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1997 1998 1999] TEST: [  18   23   28   36   39   43   46   48   52   54   57   78   80   87
  104  105  108  113  114  117  128  131  139  142  144  153  156  168
  174  178  179  180  183  184  186  189  190  195  201  206  209  210
  231  236  241  254  255  260  268  269  273  276  277  282  284  287
  288  296  303  304  321  327  340  344  369  381  382  385  387  389
  396  399  401  407  415  421  426  439  444  449  452  453  459  464
  470  472  475  485  486  487  491  499  504  505  507  509  510  515
  519  524  535  538  550  556  560  562  568  569  573  575  578  584
  586  588  589  592  606  611  614  630  635  637  638  649  656  657
  658  660  662  664  668  671  677  681  684  685  691  692  704  706
  716  719  728  732  737  744  748  761  765  766  767  769  770  771
  774  776  783  789  795  799  803  805  829  831  838  839  851  852
  853  856  863  868  874  877  883  904  906  908  909  913  918  926
  948  951  953  961  962  9

W0819 08:30:32.587787  3912 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 25s 16ms/sample - loss: 0.6358 - acc: 0.1239
Epoch 2/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.6237 - acc: 0.1206
Epoch 3/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.6136 - acc: 0.1290
Epoch 4/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5909 - acc: 0.1420
Epoch 5/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5674 - acc: 0.1548
Epoch 6/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5502 - acc: 0.1620
Epoch 7/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5439 - acc: 0.1641
Epoch 8/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5233 - acc: 0.1745
Epoch 9/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5165 - acc: 0.1773
Epoch 10/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2728 - acc: 0.2623
Epoch 81/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2694 - acc: 0.2634
Epoch 82/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2686 - acc: 0.2637
Epoch 83/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2651 - acc: 0.2650
Epoch 84/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2658 - acc: 0.2649
Epoch 85/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2636 - acc: 0.2654
Epoch 86/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2628 - acc: 0.2656
Epoch 87/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2611 - acc: 0.2664
Epoch 88/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2580 - acc: 0.2674
Epoch 89/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.25

...

[[   41   260    48   189     2   583    40   118]
 [    1 15089   191  2821    25  3468   193   827]
 [    0   410   840   869     6   986    80   571]
 [    0  2856   410 23902   135  3049   175  1551]
 [    0    49    11   222   161    50     5    35]
 [   12  3492   412  2835    48 10689   627  1925]
 [    0  1258   206  1187    20  3581  1024  1416]
 [    2  1197   340  2335    15  2649   514  4363]]
TRAIN: [   1    3    4 ... 1996 1997 1998] TEST: [   0    2   12   13   19   21   24   26   30   37   51   61   66   94
   98   99  101  106  118  119  129  130  132  133  134  135  137  143
  147  149  151  157  161  162  185  187  188  203  214  215  216  219
  220  224  234  235  239  244  251  262  265  270  272  280  285  286
  289  294  300  314  317  318  325  329  330  345  346  351  356  358
  363  365  370  378  380  386  391  392  393  394  395  398  413  418
  429  435  436  443  448  450  454  457  462  471  473  476  482  500
  502  528  534  539  540  541  546  557  561

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2892 - acc: 0.2597
Epoch 57/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2872 - acc: 0.2604
Epoch 58/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2842 - acc: 0.2614
Epoch 59/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2860 - acc: 0.2607
Epoch 60/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2805 - acc: 0.2626
Epoch 61/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2765 - acc: 0.2639
Epoch 62/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2737 - acc: 0.2649
Epoch 63/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2725 - acc: 0.2652
Epoch 64/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2702 - acc: 0.2661
Epoch 65/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.26

1600/1600 [==============================] - 22s 14ms/sample - loss: 0.5219 - acc: 0.1749
Epoch 5/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.5152 - acc: 0.1777
Epoch 6/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.5089 - acc: 0.1805
Epoch 7/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.5021 - acc: 0.1833
Epoch 8/100
1600/1600 [==============================] - 22s 13ms/sample - loss: 0.5009 - acc: 0.1838
Epoch 9/100
1600/1600 [==============================] - 22s 13ms/sample - loss: 0.4929 - acc: 0.1875
Epoch 10/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.4929 - acc: 0.1872
Epoch 11/100
1600/1600 [==============================] - 22s 13ms/sample - loss: 0.4877 - acc: 0.1897
Epoch 12/100
1600/1600 [==============================] - 22s 13ms/sample - loss: 0.4827 - acc: 0.1914
Epoch 13/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.4777 - 

1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2383 - acc: 0.2738
Epoch 84/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2396 - acc: 0.2734
Epoch 85/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2377 - acc: 0.2741
Epoch 86/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2354 - acc: 0.2752
Epoch 87/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2325 - acc: 0.2761
Epoch 88/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2345 - acc: 0.2755
Epoch 89/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2336 - acc: 0.2754
Epoch 90/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2305 - acc: 0.2768
Epoch 91/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2295 - acc: 0.2774
Epoch 92/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.22

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3772 - acc: 0.2308
Epoch 32/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3713 - acc: 0.2324
Epoch 33/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3669 - acc: 0.2338
Epoch 34/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3617 - acc: 0.2359
Epoch 35/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3610 - acc: 0.2360
Epoch 36/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3530 - acc: 0.2384
Epoch 37/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3499 - acc: 0.2391
Epoch 38/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3455 - acc: 0.2407
Epoch 39/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3406 - acc: 0.2418
Epoch 40/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.33

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_32 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_32 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_33 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_33 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_34 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_35 (Bidirectio (None, 700, 200)         

In [10]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total
[0.3940936791975221, 0.3753872389035599, 0.38486298867640384, 0.38301443428267007, 0.38427417981596257]
0.3843265041752237
Precision total
[0.5697194021095976, 0.5489025363495098, 0.5504736874946211, 0.5527866456165093, 0.5600337124993879]
0.5563831968139251
Recalls total
[0.5762026616946185, 0.5587655353728489, 0.5627318801358068, 0.5642225246682545, 0.5684673122545016]
0.566077982825206
F1 total
[0.5620931482303724, 0.5413461088269991, 0.5480453520110679, 0.5519131625998394, 0.5573231146965777]
0.5521441772729713
